In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
%matplotlib inline

In [59]:
#Laden des Versuchsplan
sheet='Tabelle1'
file_path= 'info_versuche.xlsx' 
info = pd.read_excel(file_path, sheet_name=sheet)

info

,Proben-nummer,Material,Höhe,Position,Kraft,Temp.,Werkzeug-einbau
0,1,X20CrMoV12-1,40,schräg,880,850,korrekt
1,2,16MnCr5,40,schräg,880,850,korrekt
2,3,16MnCr5,38,schräg,880,850,korrekt
3,4,16MnCr5,40,rechts,880,850,korrekt
4,5,16MnCr5,40,mittig,880,850,korrekt
...,...,...,...,...,...,...,...
643,644,X20CrMoV12-1,42,rechts,1210,1200,falsch
644,645,X20CrMoV12-1,42,mittig,1210,1200,falsch
645,646,16MnCr5,42,mittig,1210,1200,falsch
646,647,16MnCr5,38,rechts,1210,1200,falsch


In [76]:
#Erstellen der Kategorie Bad or good
info['good_or_bad']= np.ones(648)


info.loc[info['Höhe']!= 40.0,'good_or_bad'] =-1
info.loc[info['Material']!= 'C45','good_or_bad'] =-1
info.loc[info['Position']!= 'mittig','good_or_bad'] =-1
info.loc[info['Werkzeug-einbau']!= 'korrekt','good_or_bad'] =-1
info.loc[info['Temp.']!= 1200.0,'good_or_bad'] =-1
info.loc[info['Kraft']!= 1210.0,'good_or_bad'] =-1
info[info['good_or_bad']==1]


,Proben-nummer,Material,Höhe,Position,Kraft,Temp.,Werkzeug-einbau,good_or_bad
99,100,C45,40,mittig,1210,1200,korrekt,1.0
300,301,C45,40,mittig,1210,1200,korrekt,1.0
522,523,C45,40,mittig,1210,1200,korrekt,1.0


In [82]:
#Index der gut Schmiedeteile
good=info['good_or_bad']
good[good==1]

99     1.0
300    1.0
522    1.0
Name: good_or_bad, dtype: float64

In [28]:
# Erstellen der Paths zum Laden
l_or_r= 'l'
data_path= os.path.abspath(os.getcwd()) + '/crop/'
bad_path= os.path.abspath(os.getcwd()) + "/good_or_bad/bad/" 
good_path= os.path.abspath(os.getcwd()) + "/good_or_bad/good/" 


In [86]:
#Laden der Bilder und zu Ordnung zu den Klassen
n_g=45
n_b=645
shift=690
for i in range(648):
    try:
        name = data_path + "crop" + str(i+1+shift) + ".png"
        img = cv2.imread( name )
        if good[i] == 1 :
            cv2.imwrite(good_path + 'good' + str(n_g) + '.png',img )
            n_g=n_g+1
        else:
            cv2.imwrite(bad_path + 'bad' + str(n_b) + '.png',img )
            n_b=n_b+1
    except:
        print(i)
        read = False
    del img

for i in range(648+shift,690+shift): 
    try:
        name = data_path + "crop" + str(i+1) + ".png"
        img = cv2.imread( name )
        cv2.imwrite(good_path + 'good' + str(n_g) + '.png',img )
        n_g=n_g+1

    except:
        print(i)
        
    del img
    

145
1368


In [12]:
#Oversamplen der Gut teile
nt=10

for i in range(nt):
    for l in range(88):
        #try:
        s = "C:/LUH/Master/Masterarbeit/Stereo_camera_program/Daten/good_or_bad/good/good" + str(l+1) + ".png"
        t = "C:/LUH/Master/Masterarbeit/Stereo_camera_program/Daten/good_or_bad_oversampled/good/good" + str(l+1+88*i) + ".png"
        img = cv2.imread( s )
        cv2.imwrite(t, img)
        #except:
            #print(i)

In [121]:
#position  mittig 1 schräg 2 rchts 3
info['pos']= np.zeros(648)
info.loc[info['Position'] == 'mittig','pos'] = 1
info.loc[info['Position']== 'schräg','pos'] = 2
info.loc[info['Position']== 'rechts','pos'] = 3
#
info


,Proben-nummer,Material,Höhe,Position,Kraft,Temp.,Werkzeug-einbau,good_or_bad,pos
0,1,X20CrMoV12-1,40,schräg,880,850,korrekt,-1.0,2.0
1,2,16MnCr5,40,schräg,880,850,korrekt,-1.0,2.0
2,3,16MnCr5,38,schräg,880,850,korrekt,-1.0,2.0
3,4,16MnCr5,40,rechts,880,850,korrekt,-1.0,3.0
4,5,16MnCr5,40,mittig,880,850,korrekt,-1.0,1.0
...,...,...,...,...,...,...,...,...,...
643,644,X20CrMoV12-1,42,rechts,1210,1200,falsch,-1.0,3.0
644,645,X20CrMoV12-1,42,mittig,1210,1200,falsch,-1.0,1.0
645,646,16MnCr5,42,mittig,1210,1200,falsch,-1.0,1.0
646,647,16MnCr5,38,rechts,1210,1200,falsch,-1.0,3.0


In [118]:

pos=info['pos'].values
pos=pos.astype(int)
pos_all =np.append(pos, np.ones(41))
pos_all =np.append(pos_all, pos)
pos_all =np.append(pos_all, np.ones(41))
pos=pos_all.astype(int)
len(pos)

1378

In [119]:
#Zuordnen der Bilder zu Positionen
n_m=1
n_s=1
n_r=1

for i in range(len(pos)):
    try:
        name = data_path + "crop" + str(i+1) + ".png"
        img = cv2.imread( name )

        if pos[i] == 1 :
            cv2.imwrite(os.path.abspath(os.getcwd()) + "/Halbzeug_Position/mittig/"  + str(n_m) + '.png',img )
            n_m=n_m+1
            print('m'+ str(i))
        if pos[i] == 2 :
            cv2.imwrite(os.path.abspath(os.getcwd()) + "/Halbzeug_Position/schraeg/"   + str(n_s) + '.png',img )
            n_s=n_s+1
            print('s'+ str(i))
        if pos[i] == 3 :
            cv2.imwrite(os.path.abspath(os.getcwd()) + "/Halbzeug_Position/rechts/"   + str(n_r) + '.png',img )
            n_r=n_r+1
            print('r' + str(i))
        
    
    except:
        print(i)
    del img

s0
s1
s2
r3
m4
r5
s6
m7
m8
r9
m10
s11
m12
r13
r14
m15
s16
s17
m18
m19
r20
r21
s22
s23
r24
r25
m26
m27
m28
m29
s30
s31
s32
m33
r34
m35
r36
r37
s38
r39
r40
r41
s42
r43
s44
m45
s46
r47
s48
s49
m50
m51
m52
r53
r54
m55
s56
m57
r58
m59
m60
m61
m62
r63
s64
s65
s66
r67
r68
r69
s70
s71
s72
r73
m74
m75
r76
m77
s78
r79
s80
m81
r82
m83
s84
m85
r86
m87
m88
r89
s90
r91
s92
r93
r94
s95
m96
m97
s98
m99
r100
s101
s102
s103
m104
r105
r106
s107
m108
s109
r110
r111
s112
s113
r114
r115
r116
r117
m118
s119
r120
m121
s122
m123
m124
r125
s126
m127
s128
m129
r130
s131
m132
m133
s134
m135
r136
r137
m138
s139
m140
r141
s142
m143
s144
145
m146
s147
s148
m149
s150
r151
s152
r153
r154
r155
r156
s157
m158
r159
s160
m161
s162
r163
m164
r165
r166
s167
r168
m169
m170
m171
r172
s173
r174
s175
m176
m177
s178
m179
r180
m181
m182
r183
s184
r185
s186
s187
s188
m189
r190
m191
s192
m193
s194
r195
r196
r197
r198
s199
r200
s201
m202
m203
m204
m205
s206
m207
m208
s209
r210
r211
s212
r213
s214
s215
m216
r217
r218
s219
s220
s221
r

In [122]:
#werkzeug
info['werk']= np.zeros(648)
info.loc[info['Werkzeug-einbau'] == 'korrekt','werk'] = 1
info.loc[info['Werkzeug-einbau']== 'falsch','werk'] = 2

#
info

,Proben-nummer,Material,Höhe,Position,Kraft,Temp.,Werkzeug-einbau,good_or_bad,pos,werk
0,1,X20CrMoV12-1,40,schräg,880,850,korrekt,-1.0,2.0,1.0
1,2,16MnCr5,40,schräg,880,850,korrekt,-1.0,2.0,1.0
2,3,16MnCr5,38,schräg,880,850,korrekt,-1.0,2.0,1.0
3,4,16MnCr5,40,rechts,880,850,korrekt,-1.0,3.0,1.0
4,5,16MnCr5,40,mittig,880,850,korrekt,-1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
643,644,X20CrMoV12-1,42,rechts,1210,1200,falsch,-1.0,3.0,2.0
644,645,X20CrMoV12-1,42,mittig,1210,1200,falsch,-1.0,1.0,2.0
645,646,16MnCr5,42,mittig,1210,1200,falsch,-1.0,1.0,2.0
646,647,16MnCr5,38,rechts,1210,1200,falsch,-1.0,3.0,2.0


In [123]:
werk=info['werk'].values
werk=werk.astype(int)
werk_all =np.append(werk, np.ones(41))
werk_all =np.append(werk_all, werk)
werk_all =np.append(werk_all, np.ones(41))
werk=werk_all.astype(int)
len(werk)

1378

In [126]:
n_k=1
n_f=1


for i in range(len(werk)):
    try:
        name = data_path + "crop" + str(i+1) + ".png"
        img = cv2.imread( name )

        if werk[i] == 1 :
            cv2.imwrite(os.path.abspath(os.getcwd()) + "/Werkzeug_verbaut/korrekt/"  + str(n_k) + '.png',img )
            n_k=n_k+1
            
        if werk[i] == 2 :
            cv2.imwrite(os.path.abspath(os.getcwd()) + "/Werkzeug_verbaut/falsch/"   + str(n_f) + '.png',img )
            n_f=n_f+1
            

        
    
    except:
        print(i)
    del img

145
678
835
1368
